# LAB 5: Data Generation, Density Recenstruction
### Parzen-Rosenblatt, LOO, EM Algorithms
### AI and Machine Learning // Suchkova Natalia М8О-114М-22
20.12.2022 @ MAI IT-Center

### Task
+ реализовать 3 генератора данных на плоскости, прямой или в пространстве (равномерное, Гаусс.распр, ... и др)
+ написать реализацию метода Парзена-Розенблатта
+ восстановить плотность в виде картинки
+ для 1 из 3 наборов данных (п.1) применить оптимизацию ширины окна методом LOO
+ для сгенерированных данных с 2-мя и более горбами применить EM-алгоритм (Gaussian Mixture) из sckit-learn